In [115]:
np.unique(preds, return_counts=True)

(array(['hate', 'none', 'offensive'], dtype=object),
 array([ 77, 274, 120], dtype=int64))

In [119]:
import pandas as pd
from konlpy.tag import Okt; t = Okt()
from sklearn.feature_extraction.text import TfidfVectorizer

In [120]:
train = pd.read_csv('./datas/train_ver1', index_col=[0])
train.tail()

,comments,bias_label,gender_label,hate_label,news_title,comment_pos,title_pos
7891,"힘내세요,응원합니다",none,False,none,"허지웅 허투루 넘길 말 없었다,솔직하게 드러냈던 속 사정","[('힘내세요', 'Verb'), (',', 'Punctuation'), ('응원'...","[('허지웅', 'Noun'), ('허투루', 'Noun'), ('넘길', 'Ver..."
7892,"힘내세요,삼가 고인의 명복을 빕니다",none,False,none,이혜경 오 캐롤 공연 중 남편 오정욱 부 고 오열 속 발인 종합,"[('힘내세요', 'Verb'), (',', 'Punctuation'), ('삼가'...","[('이혜경', 'Noun'), ('오', 'Noun'), ('캐롤', 'Noun'..."
7893,힘내세용 항상 응원합니닷,none,False,none,설경구 송윤아 아들과 즐거운 하루 전 엄마니까요,"[('힘내세용', 'Verb'), ('항상', 'Noun'), ('응원', 'Nou...","[('설경구', 'Noun'), ('송윤아', 'Noun'), ('아들', 'Nou..."
7894,"힘내 소연기로 답해요,나도 53살 인데 이런 일 저런 일 다 있더라구 요인격을 믿습...",none,False,none,SC 현장 연예인 인생 협박 유감 미소 잃은 최민수 보복운전 혐의 2차 공판 종합,"[('힘내', 'Verb'), ('소', 'Modifier'), ('연기', 'No...","[('SC', 'Alpha'), ('현장', 'Noun'), ('연예인', 'Nou..."
7895,힘들면 관뒀어야지 그게 현명한 거다,none,False,none,단독 스태프 사망 사고 서른 이지만 결국 오늘 촬영 취소,"[('힘들면', 'Adjective'), ('관', 'Noun'), ('뒀어야지',...","[('단독', 'Noun'), ('스태프', 'Noun'), ('사망', 'Noun..."


In [121]:
dev = pd.read_csv('./datas/dev.hate.csv')
dev.rename(columns={'label': 'hate_label'}, inplace=True)
dev.tail()

,comments,hate_label
466,지현우 범죄 저지르지 않았나요?,offensive
467,여자인생 망칠 일 있나 ㅋㅋ,hate
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,offensive
469,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",hate
470,남자가 잘못한거라면... 반성도 없다면...나였다면 ... 여자처럼 아주 못되게 할...,none


In [122]:
def t_tokenizer(text):
    tokens_ko = t.morphs(text)
    return tokens_ko

In [123]:
stopwords = ['...', '....', 'ㅠㅠ', 'ㅜ', 'ㅎㅎ', ',,,', '^^', '!!', '!!!', 'ㅡㅡ', 'ㅋㅋㅋ', 'ㅋ']

In [125]:
tfidf_vect = TfidfVectorizer(tokenizer=t_tokenizer, stop_words=stopwords, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train['comments'])
tfidf_matrix_train = tfidf_vect.transform(train['comments'])

## LGBMClassifier compare train and dev data

In [126]:
from lightgbm import LGBMClassifier
import time

start_time = time.time()
lgbm_clf = LGBMClassifier(n_estimators=400)
lgbm_clf.fit(tfidf_matrix_train, train['hate_label'])
print('fit time:', time.time() - start_time)

fit time: 9.823092460632324


In [127]:
from sklearn.metrics import f1_score, accuracy_score, recall_score

tfidf_matrix_dev = tfidf_vect.transform(dev['comments'])
preds = lgbm_clf.predict(tfidf_matrix_dev)
accuracy_score(dev['hate_label'], preds), f1_score(preds, dev['hate_label'], average='macro')

(0.5498938428874734, 0.542144386766068)

## DecisionTreeClassifier compare train and dev data

In [128]:
from sklearn.tree import DecisionTreeClassifier

decision_clf = DecisionTreeClassifier(max_depth=2, random_state=13)
decision_clf.fit(tfidf_matrix_train, train['hate_label'])
print('fit time:', time.time() - start_time)

fit time: 12.927754878997803


In [129]:
preds = decision_clf.predict(tfidf_matrix_dev)
accuracy_score(dev['hate_label'], preds), f1_score(preds, dev['hate_label'], average='macro')

(0.4118895966029724, 0.33302547396349347)

## LogisticRegression compare train and dev data

In [130]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='liblinear', random_state= 13)
lr.fit(tfidf_matrix_train, train['hate_label'])
print('fit time:', time.time() - start_time)

fit time: 19.302770853042603


In [131]:
preds = lr.predict(tfidf_matrix_dev)
accuracy_score(dev['hate_label'], preds), f1_score(preds, dev['hate_label'], average='macro')

(0.5881104033970276, 0.5775181595719191)

In [132]:
len(preds)

471

In [133]:
wrong_result = []

for n in range(0, len(dev['hate_label'])):
    if preds[n] != dev['hate_label'][n]:
        wrong_result.append(n)
len(wrong_result)

194

In [134]:
import random

samples = random.choices(population=wrong_result, k=300)
samples

[232,
 442,
 212,
 157,
 279,
 263,
 16,
 226,
 446,
 308,
 31,
 358,
 239,
 303,
 166,
 248,
 255,
 346,
 470,
 73,
 36,
 387,
 380,
 345,
 325,
 119,
 127,
 271,
 30,
 263,
 303,
 43,
 403,
 35,
 15,
 37,
 442,
 301,
 217,
 303,
 223,
 426,
 269,
 403,
 314,
 359,
 293,
 353,
 88,
 172,
 224,
 293,
 219,
 173,
 312,
 438,
 258,
 403,
 301,
 114,
 119,
 103,
 255,
 235,
 55,
 113,
 37,
 351,
 71,
 330,
 426,
 73,
 281,
 403,
 263,
 431,
 425,
 164,
 470,
 303,
 435,
 399,
 341,
 250,
 66,
 179,
 36,
 30,
 115,
 61,
 19,
 393,
 49,
 401,
 434,
 122,
 367,
 66,
 370,
 46,
 111,
 92,
 181,
 232,
 184,
 433,
 220,
 351,
 147,
 434,
 2,
 19,
 374,
 273,
 209,
 258,
 394,
 41,
 387,
 50,
 55,
 69,
 98,
 436,
 16,
 101,
 16,
 223,
 83,
 259,
 15,
 225,
 302,
 367,
 92,
 209,
 425,
 401,
 156,
 29,
 141,
 37,
 219,
 241,
 324,
 442,
 70,
 10,
 401,
 431,
 29,
 308,
 330,
 163,
 394,
 217,
 366,
 359,
 220,
 44,
 458,
 322,
 297,
 293,
 273,
 393,
 331,
 50,
 199,
 119,
 367,
 337,
 138,
 367,

In [135]:
df = pd.DataFrame(preds[wrong_result])
df

,0
0,offensive
1,none
2,offensive
3,offensive
4,none
...,...
189,none
190,none
191,none
192,none


In [136]:
df1 = dev.iloc[wrong_result]
df1['error'] = preds[wrong_result]
df1

,comments,hate_label,error
2,알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라,hate,offensive
3,설마 ㅈ 현정 작가 아니지??,hate,none
10,이경규가 이런거보면 세련되긴함 저 나이에 차은우 누가 알꼬 아무리 잘생겼다해도. 배...,none,offensive
11,아c발 어쩌라고 뭔기사가계속나오냐,hate,offensive
15,둘이 화장실가서 싸우길,offensive,none
...,...,...,...
463,강용석도 찌질하네 과거들추기는 추하다,offensive,none
466,지현우 범죄 저지르지 않았나요?,offensive,none
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,offensive,none
469,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",hate,none


In [118]:
df1.to_csv('./datas/df1_wrong_result', index=False)

In [113]:
df1['hate_label'].value_counts().to_frame()

,hate_label
offensive,116
hate,61
none,14


In [114]:
df1['error'].value_counts().to_frame()

,error
none,128
offensive,47
hate,16


In [117]:
np.unique(preds, return_counts=True)

(array(['hate', 'none', 'offensive'], dtype=object),
 array([ 77, 274, 120], dtype=int64))

In [138]:
test = pd.read_csv('./datas/NaiveBayeji_prediction.csv')
test

,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,0
1,둘다 넘 좋다~행복하세요,0
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,0
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
4,장현승 얘도 참 이젠 짠하다...,1
...,...,...
969,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,0
970,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,1
971,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,0
972,입에 손가릭이 10개 있으니 징그럽다,2


## GradientBoostingClassifier compare train and dev data

In [25]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(random_state=13)
gb_clf.fit(tfidf_matrix_train, train['hate_label'])
print('fit time:', time.time() - start_time)

fit time: 20.991591930389404


In [26]:
preds = gb_clf.predict(tfidf_matrix_dev)
accuracy_score(dev['hate_label'], preds), f1_score(preds, dev['hate_label'], average='macro')

(0.5095541401273885, 0.48810533169257236)

## XGBClassifier compare train and dev data

In [28]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
xgb_clf.fit(tfidf_matrix_train, train['hate_label'])
print('fit time:', time.time() - start_time)

[21:21:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fit time: 31.689951181411743


In [29]:
preds = xgb_clf.predict(tfidf_matrix_dev)
accuracy_score(dev['hate_label'], preds), f1_score(preds, dev['hate_label'], average='macro')

(0.5265392781316348, 0.506364447945686)

In [31]:
## RandomForestClassifier compare train and dev data

In [30]:
from sklearn.ensemble import RandomForestClassifier

r_clf = RandomForestClassifier(max_depth=4, random_state=13)
r_clf.fit(tfidf_matrix_train, train['hate_label'])

preds = r_clf.predict(tfidf_matrix_dev)
accuracy_score(dev['hate_label'], preds), f1_score(preds, dev['hate_label'], average='macro')

(0.33970276008492567, 0.16904384574749076)

In [59]:
from sklearn.model_selection import train_test_split

msg_train, msg_test, class_train, class_test = train_test_split(tfidf_matrix_train, train['hate_label'], test_size=0.2, stratify=train['hate_label'])

In [60]:
from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

models = []
models.append(('RandomForestClassifier', RandomForestClassifier()))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
models.append(('AdaBoostClassifier', AdaBoostClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
models.append(('LogisticRegression', LogisticRegression()))
models.append(('LGBMClassifier', LGBMClassifier()))

In [61]:
from sklearn.metrics import f1_score, accuracy_score, recall_score

for name, model in models:
    clf = model
    clf.fit(msg_train, class_train)
    
    y_pred_tr = clf.predict(msg_train)
    y_pred_test = clf.predict(msg_test)
    
    print('Train Acc : ', accuracy_score(class_train, y_pred_tr))
    print('Test Acc : ', accuracy_score(class_test, y_pred_test))

Train Acc :  0.9988917036098797
Test Acc :  0.5360759493670886
Train Acc :  0.9988917036098797
Test Acc :  0.46392405063291137
Train Acc :  0.5582647245091831
Test Acc :  0.510759493670886
Train Acc :  0.652786573780874
Test Acc :  0.5259493670886076
Train Acc :  0.8530715642811906
Test Acc :  0.5430379746835443
Train Acc :  0.8096896770107663
Test Acc :  0.5316455696202531


In [62]:
train_score = []
test_score = []
names = []
f1score = []

for name, model in models:
    clf = model
    clf.fit(msg_train, class_train)
    
    y_pred_tr = clf.predict(msg_train)
    y_pred_test = clf.predict(msg_test)
    
    names.append(name)
    test_score.append(accuracy_score(class_test, y_pred_test))
    train_score.append(accuracy_score(class_train, y_pred_tr))
    f1score.append(f1_score(class_test, y_pred_test, average='macro'))

In [63]:
result = pd.DataFrame({'model name':names, 
                       'train score':train_score, 
                       'test score':test_score,
                       'f1 score': f1score}) 
result

,model name,train score,test score,f1 score
0,RandomForestClassifier,0.998892,0.532911,0.474092
1,DecisionTreeClassifier,0.998892,0.473418,0.441096
2,AdaBoostClassifier,0.558265,0.510759,0.444480
3,GradientBoostingClassifier,0.652787,0.524684,0.451602
4,LogisticRegression,0.853072,0.543038,0.507301
5,LGBMClassifier,0.809690,0.531646,0.498601
